<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Veracity75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import fastai

In [0]:
from fastai.text import * 

In [0]:
def read(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 3.0:
            return 'Query'
        elif col['label'] == 2.0:
            return 'Deny'
        elif col['label'] == 1.0:
            return 'Support'
        elif col['label'] == 0.0:
            return 'Comment'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
train = read('train.csv', True)
test = read('test.csv')

In [0]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [0]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [0]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.115868,4.618284,0.311384,00:00


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.798488,3.841085,0.357812,00:00


In [0]:
learn.predict("This is a review about", n_words=10)

'This is a review about the " Islamic War " it was at'

In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (217 items)
x: TextList
xxbos xxmaj canada 's parliament building locked down after shooting ; at least one guard injured : http : / / t.co / xxmaj xxunk xxmaj photo : xxup ap http : / / t.co / xxunk,xxbos # sydneysiege : xxmaj latest video of hostages xxunk . 
  https : / / t.co / xxunk,xxbos xxmaj is # xxmaj prince in # xxmaj toronto ? xxmaj rumours xxunk of surprise show xxunk xxunk http : / / t.co / xxunk,xxbos xxup breaking : a source xxunk to xxup xxunk that the xxunk soldier in xxmaj ottawa is xxmaj cpl . xxmaj nathan xxmaj cirillo . xxmaj cirillo was a xxunk of the xxmaj hamilton xxmaj xxunk,xxbos xxmaj coup in # xxmaj russia ? xxmaj good xxunk by xxunk . http : / / t.co / xxunk # xxunk # xxmaj putin # putindead http : / / t.co / xxunk
y: CategoryList
Deny,Deny,Comment,Support,Deny
Path: .;

Valid: LabelList (55 items)
x: TextList
xxbos xxmaj france : 10 people dead after shooting at xxup hq of satirical weekly newspaper # ch

In [0]:
data_clas.show_batch()

text,target
"xxbos xxmaj name of # xxmaj ferguson cop is xxunk to be released by xxunk xxup et , police chief says : http : / / t.co / xxunk ( xxup xxunk xxmaj xxunk , xxup ap ) http : / / t.co / xxunk",Deny
"xxbos xxmaj xxunk like xxunk xxmaj charlie xxmaj hebdo xxunk has been killed : 1 . xxmaj charb - was on xxmaj al - xxmaj xxunk 's xxunk xxunk ; 2 . xxmaj cabu , 3 . xxmaj xxunk , 4 . xxmaj xxunk",Deny
xxbos xxmaj xxunk unarmed kid . \n xxmaj xxunk xxunk . \n xxmaj xxunk xxunk xxunk . \n xxmaj xxunk xxunk . \n xxmaj xxunk the xxunk . \n xxmaj xxunk . xxmaj police . xxmaj xxunk . \n # xxmaj ferguson # mikebrown,Deny
"xxbos xxup sydney xxup hostage - xxup taker \n - xxmaj man xxmaj monis , xxunk \n - xxmaj xxunk from xxmaj iran \n - xxmaj xxunk - xxunk xxunk \n - xxmaj xxunk of xxunk xxunk xxunk xxunk \n xxup developing ..",Support
"xxbos xxmaj nearly xxunk blacks were murdered last xxunk -- xxunk all by other blacks . a xxunk xxunk were unarmed - killed - by - cop . \n xxmaj where 's xxmaj al , xxmaj xxunk ? \n # tcot # ferguson",Deny


In [0]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.191295,1.075609,0.400000,00:00


In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.990523,1.042854,0.490909,00:00


In [0]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.872093,1.015845,0.436364,00:00
1,0.834005,0.954650,0.527273,00:00
2,0.780148,0.857889,0.600000,00:00
3,0.738507,0.807178,0.672727,00:00
4,0.665777,0.765118,0.672727,00:00
5,0.605861,0.759656,0.745455,00:00
6,0.538210,0.781290,0.690909,00:00
7,0.471227,0.787271,0.690909,00:00
8,0.423149,0.788311,0.690909,00:00
9,0.377620,0.782522,0.690909,00:00


In [0]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [0]:
accuracy_score(test['label'], l)

0.75

In [0]:
f1_score(test['label'], l, average='macro')

0.7481481481481481

In [0]:
learn.export('veracity.pkl')